In [128]:
import json
import os
import scrap_page_joueur as spj
from requests import get
from bs4 import BeautifulSoup
from dataclasses import dataclass
from typing import List

# Récupérer les liens des matchs 

In [6]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
print(current_dir,parent_dir)

d:\Cours_M2\webscraping\ml-webscrap-tennis\scraping d:\Cours_M2\webscraping\ml-webscrap-tennis


In [10]:
path_detail_joueurs = os.path.join(parent_dir, "donnees", "detail_joueurs.json")
with open(path_detail_joueurs, "r") as fichier:
    detail_joueurs = json.load(fichier)

In [50]:
liens_match = set()
for key, value in detail_joueurs.items():
    joueur = detail_joueurs[key]
    matchs = joueur['matchs']
    
    for i in range(len(matchs)):
        lien_match = matchs[i]['lien_detail_match']
        liens_match.add(lien_match)

In [52]:
liens_match

{'https://www.tennisendirect.net/atp/match/alejandro-davidovich-fokina-VS-daniil-medvedev/national-bank-open-montreal-2024/',
 'https://www.tennisendirect.net/atp/match/alejandro-tabilo-VS-novak-djokovic/internazionali-bnl-ditalia-rome-2024/',
 'https://www.tennisendirect.net/atp/match/aleksandar-vukic-VS-casper-ruud/shanghai-rolex-masters-shanghai-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-alex-michelsen/french-open-paris-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-alexander-bublik/bnp-paribas-open-indian-wells-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-alexei-popyrin/monte-carlo-rolex-masters-monte-carlo-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-arthur-fils/wimbledon-london-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-daniel-evans/us-open-new-york-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-daniil-medvedev/french-open-paris-2024/',
 

# Test de scrap sur un match : match_1

In [55]:
matchs = list(liens_match)
match_1 = matchs[0]
match_1

'https://www.tennisendirect.net/atp/match/holger-vitus-nodskov-rune-VS-alex-de-minaur/rolex-paris-masters-paris-2024/'

## Récupération des stats des 2 joueurs sur le match 

In [130]:
@dataclass
class StatsMatch:
    nom_joueur: str
    premier_service: str
    pnts_gagnes_ps: str
    pnts_gagnes_ss: str
    balles_break_gagnees: str
    retours_gagnes: str
    total_points_gagnes: str
    double_fautes: str
    aces: str

In [131]:
def extraire_colonnes(table: BeautifulSoup) -> List[str]:
    """
    Extrait les données des colonnes sous forme de liste de texte brut.
    """
    return [td.text.strip() for td in table.find_all("td")]

In [132]:
def organiser_stats_par_joueur(table_data: List[str]) -> tuple:
    """
    Réorganise les données extraites en deux listes, une pour chaque joueur.
    """
    joueur_A, joueur_B = [], []
    for i, data in enumerate(table_data):
        if i % 3 == 1:  # Colonne du joueur A
            joueur_A.append(data)
        elif i % 3 == 2:  # Colonne du joueur B
            joueur_B.append(data)
    return joueur_A, joueur_B

In [133]:
def creer_stats_joueur(joueur_data: List[str]) -> StatsMatch:
    """
    Crée une instance de StatsMatch pour un joueur à partir des données extraites.
    """
    return StatsMatch(
        nom_joueur=joueur_data[0],
        premier_service=joueur_data[1],
        pnts_gagnes_ps=joueur_data[2],
        pnts_gagnes_ss=joueur_data[3],
        balles_break_gagnees=joueur_data[4],
        retours_gagnes=joueur_data[5],
        total_points_gagnes=joueur_data[6],
        double_fautes=joueur_data[7],
        aces=joueur_data[8],
    )

In [134]:
reponse = get(match_1)
assert reponse.status_code == 200, f"Erreur HTTP {reponse.status_code}: {match_1}"

In [135]:
detail_match = BeautifulSoup(reponse.text, features="lxml")
statistiques = detail_match.find_all("table", attrs={"class": "table_stats_match"})[0]

In [136]:
table_data = extraire_colonnes(statistiques)
joueur_A_data, joueur_B_data = organiser_stats_par_joueur(table_data)

In [137]:
stats_joueur_A = creer_stats_joueur(joueur_A_data)
stats_joueur_B = creer_stats_joueur(joueur_B_data)

print(stats_joueur_A)
print(stats_joueur_B)

StatsMatch(nom_joueur='Holger Rune', premier_service='58/87 (67%)', pnts_gagnes_ps='46/58 (79%)', pnts_gagnes_ss='15/29 (52%)', balles_break_gagnees='3/9 (33%)', retours_gagnes='33/94 (35%)', total_points_gagnes='94/181 (52%)', double_fautes='1', aces='5')
StatsMatch(nom_joueur='Alex De Minaur', premier_service='44/94 (47%)', pnts_gagnes_ps='34/44 (77%)', pnts_gagnes_ss='27/50 (54%)', balles_break_gagnees='2/4 (50%)', retours_gagnes='26/87 (30%)', total_points_gagnes='87/181 (48%)', double_fautes='5', aces='5')
